## Deep Matrix Factorization Using Apache MXNet: The Engine for a Better Recommendation Engine

Recommendation engines are widely used models that attempt to identify items that a person will like based on that person's past behavior. We're all familiar with Amazon's recommendations based on your past purchasing history, and Netflix recommending shows to you based on your history and the ratings you've given other shows. Naturally, machine learning is behind many of these systems. In this tutorial we will delve into how to use machine learning to build these recommender systems, and specifically how to implement a technique called matrix factorization using Apache MXNet. It presumes basic familiarity with MXNet.

Matrix factorization is a linear method, meaning that if there are complicated non-linear interactions going on in the dataset, a simple dot product may not be able to handle it well. Given the recent success of deep learning in complicated non-linear computer vision and natural language processing tasks, it is natural to want to find a way to incorporate it into matrix factorization as well. A way to do this is called "deep matrix factorization" and involves the replacement of the dot product with a neural network that is trained jointly with the factors. This makes the model more powerful because a neural network can model important non-linear combinations of factors to make better predictions.

In traditional matrix factorization the prediction is the simple dot product between the factors for each of the dimensions. In contrast, in deep matrix factorization the factors for both are concatenated together and used as the input to a neural network whose output is the prediction. The parameters in the neural network are then trained jointly with the factors to produce a sophisticated non-linear model for matrix factorization. 

This tutorial will apply deep matrix factorization concepts to model the MovieLens 20M dataset. We'll explore how one can use recent advances in deep learning and the flexibility of neural networks to build more complex models. While this may sound complicated, fortunately Apache MXNet makes it easy!

In [ ]:
%pylab inline
import mxnet as mx
import pandas
import seaborn; seaborn.set_style('whitegrid')
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

The MovieLens 20M dataset is comprised of movie ratings from the MovieLens site (<a href="https://movielens.org/">https://movielens.org/</a>), a site that will predict what other movies you will like after seeing you rate movies. The MovieLens 20M dataset is a sampling of ~20 million ratings from ~138 thousand users on ~27 thousand movies. The ratings range from 0.5 to 5 stars in 0.5 star increments.

<b>NOTE</b>: Given that we are about to train neural networks on millions of samples, the following cells may take some time to execute on a CPU and may produce a lot of output. At least 6GB of RAM is suggested for running the cells.  

Let's first load up the data and inspect it manually.

In [ ]:
import os
import urllib.request
import zipfile

if not os.path.exists('ml-20m.zip'):
    urllib.request.urlretrieve('http://files.grouplens.org/datasets/movielens/ml-20m.zip', 'ml-20m.zip')
with zipfile.ZipFile("ml-20m.zip", "r") as f:
    f.extractall("./")

data = pandas.read_csv('./ml-20m/ratings.csv', sep=',', usecols=(0, 1, 2))
data.head()

There are slightly over 20M ratings that comprise a user ID, a movie ID, and a rating in 0.5 star increments. Each one of these rows will be a sample to train on, as we want our model to take in a user and a movie and predict the rating that user would give to that movie. Let's take a look at the distribution of ratings before moving on.

In [ ]:
plt.hist(data['rating'])
plt.xlabel("Rating", fontsize=14)
plt.ylabel("Count", fontsize=14)
plt.title("Distribution of Ratings in MovieLens 20M", fontsize=14)
plt.show()

It looks like this is fairly normal distribution, with the most ratings being that a movie was good but not amazing, and the fewest ratings that movies were very poor. It also seems like people were more reluctant to rate a movie using one of the half star increments than with the full star increments. 

Let's next quickly look at the users and movies. Specifically, let's look at the maximum and minimum id values, and the number of unique users and movies.

In [ ]:
print("user id min/max: ", data['userId'].min(), data['userId'].max())
print("# unique users: ", numpy.unique(data['userId']).shape[0])
print("")
print("movie id min/max: ", data['movieId'].min(), data['movieId'].max())
print("# unique movies: ", numpy.unique(data['movieId']).shape[0])

It looks like the max user ID is equal to the number of unique users, but this is not the case for the number of movies. Good thing that we caught this now, otherwise we may have encountered errors if we assumed that because there are only 26744 unique movies that the maximum movie ID was 26744 as well. Let's quickly set these sizes so that later on we can set the embedding layer size appropriately.

In [ ]:
n_users, n_movies = 138493, 131262
batch_size = 25000

We can quickly estimate the sparsity of the MovieLens 20M dataset using these numbers. If there are ~138k unique users and ~27k unique movies, then there are ~3.7 billion entries in the matrix. Since only ~20M of these are present, ~99.5% of the matrix is missing. This type of massive sparsity is common in recommender systems and underlies the necessity of building models that can leverage the existing data to learn complex relationships.

Next, we need to split the data into a training set and a test set. Let's first shuffle the data to ensure that the data is randomly selected, then we can use the first 19 million samples for training and the remaining for the test set.

In [ ]:
n = 19000000

data = data.sample(frac=1).reset_index(drop=True) # Shuffle the data in place row-wise

train_users = data['userId'].values[:n] - 1 # Offset by 1
train_movies = data['movieId'].values[:n] - 1 # Offset by 1 
train_ratings = data['rating'].values[:n]

valid_users = data['userId'].values[n:] - 1
valid_movies = data['movieId'].values[n:] - 1
valid_ratings = data['rating'].values[n:]

We next need to convert our data from the form of three vectors (the row ids, column ids, and values) into an appropriate iterator that MXNet can use. Since we have multiple inputs (the movie and the user), the NDArrayIter object is the most convenient as it can handle arbitrary inputs and outputs through the use of dictionaries. 

One aspect of deep matrix factorization's flexibility with the `output_dim` size of the embedding layers. This is useful in the case where one dimension may be significantly larger than the other and thus requires training a massive number of factors. In the MovieLens case there are significantly more users (~138k) than there are movies (~27k). By changing the number of user factors from 25 to 15 we can reduce the number of parameters by 1.38 million while not losing any expressivity on the movie side. The only change is changing the value of `output_dim` in the user embedding layer.


Next we need to fit the network using the fit function, similar to scikit-learn and other popular machine learning packages. We can specify the number of epochs that we would like to train for, the validation set data through eval_data, and the metric used to evaluate both the training and validation sets during training. In this case we choose root mean squared error (rmse), which is a common choice in regression problems such as this—it penalizes large errors more than small ones. This makes sense in a recommendation context, where offering a user something slightly unexpected may be a virtue, but getting a recommendation totally wrong will be off-putting to your user.[Adam](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Adam) is a popular optimizer that can automatically tune the learning rate to get better results, and we can specify that we want to use it by adding in `optimizer='adam'` to the `fit` function.

In [ ]:
X_train = mx.io.NDArrayIter({'user': train_users, 'movie': train_movies}, 
                            label=train_ratings, batch_size=batch_size)
X_eval = mx.io.NDArrayIter({'user': valid_users, 'movie': valid_movies}, 
                           label=valid_ratings, batch_size=batch_size)

user = mx.symbol.Variable("user")
user = mx.symbol.Embedding(data=user, input_dim=n_users, output_dim=15) # Using 15 instead of 25 here

movie = mx.symbol.Variable("movie")
movie = mx.symbol.Embedding(data=movie, input_dim=n_movies, output_dim=25)

y_true = mx.symbol.Variable("softmax_label")

nn = mx.symbol.concat(user, movie)
nn = mx.symbol.flatten(nn)
nn = mx.symbol.FullyConnected(data=nn, num_hidden=64)
nn = mx.symbol.Activation(data=nn, act_type='relu') 
nn = mx.symbol.FullyConnected(data=nn, num_hidden=64)
nn = mx.symbol.Activation(data=nn, act_type='relu')
nn = mx.symbol.FullyConnected(data=nn, num_hidden=1)

y_pred = mx.symbol.LinearRegressionOutput(data=nn, label=y_true)

model = mx.module.Module(context=mx.cpu(), data_names=('user', 'movie'), symbol=y_pred)
model.fit(X_train, num_epoch=5, optimizer='adam', optimizer_params=(('learning_rate', 0.001),),
          eval_metric='rmse', eval_data=X_eval, batch_end_callback=mx.callback.Speedometer(batch_size, 250))

This technique is useful in a setting where you may run out of memory due to the size of the matrix being completed, or as a way to reduce overfitting by using a simpler model. One can imagine that perhaps movies might be difficult to appropriately represent and have many aspects that should be modeled, whereas users are simpler and only have a few features that are relevant. Being able to tune the size of these embedding layers is a very useful tool.

Next, we can extend matrix factorization past using only two embedding layers. The MovieLens dataset comes with genres for each of the films. Presumably genre is one of the major aspects that would be learned by the movie embedding layer already, so explicitly including that information can be beneficial. More broadly, perhaps there are some features that are common to a ~genre~ of films instead of an individual film. By modeling the genre of film as its own embedding layer, we can train these factors using all films of a given genre instead of trying to learn the value of the respective factor for each movie individually. Since we've already seen that the number of factors in each layer can be variable, we don't need to be concerned with the size of any of these layers. We can see that visually below. Assuming that movies have been sorted by genre, we can see that half of the movie factors now correspond more broadly to the genre that they are in and are shared across movies, while the other half are still movie specific factors. The change to the network is conceptually very simple, now concatenating three embedding layers to be the input to the network instead of only two.

<img src="images/embedding-layers.png">

Let's first load up the genre data.

In [ ]:
genres = pandas.read_csv('./ml-20m/movies.csv')
genres.head()

It looks like this dataset is simple, it just has all of the genres and the corresponding title and movieId. For simplicity, let's only use the first genre of the many that are specified, and determine a unique ID for each of the genres. Modeling all the genres applied to a movie is left as an exercise for the reader (hint: instead of label, use a one- or multiple-hot vector!).

In [ ]:
labels_str = [label.split("|")[0] for label in genres['genres']]
label_set = numpy.unique(labels_str)
label_idxs = {l: i for i, l in enumerate(label_set)}
label_idxs

It looks like there are 20 genres, with one being that no genres are listed. This seems appropriate. We want our network to now take in three numbers, the user ID, the movie ID, and the movie genre ID. This will allow us to train factors specific to all romance movies, for example, or all fantasy movies.

In [ ]:
labels = numpy.empty(n_movies)
for movieId, label in zip(genres['movieId'], labels_str):
    labels[movieId-1] = label_idxs[label]

train_genres = numpy.array([labels[int(j)] for j in train_movies])
valid_genres = numpy.array([labels[int(j)] for j in valid_movies])
train_genres[:10]

Now we have our three labels. We need to move our data iterator to take in movie type as a third input, and our network to have a third embedding layer. Let's move five factors over from the movie factors to this new movie genre embedded layer. This is another way one can attempt to structurally regularize their model, as roughly 135,000 parameters are removed from the movie embedding layer, and only 100 are added in the genre layer. The input to the network remains the same size, 40, but the values are now comprised of three layers instead of just two.

In [ ]:
X_train = mx.io.NDArrayIter({'user': train_users, 'movie': train_movies, 'movie_genre': train_genres}, 
                            label=train_ratings, batch_size=batch_size)
X_eval = mx.io.NDArrayIter({'user': valid_users, 'movie': valid_movies, 'movie_genre': valid_genres}, 
                           label=valid_ratings, batch_size=batch_size)

user = mx.symbol.Variable("user")
user = mx.symbol.Embedding(data=user, input_dim=n_users, output_dim=15)

movie = mx.symbol.Variable("movie")
movie = mx.symbol.Embedding(data=movie, input_dim=n_movies, output_dim=20) # Reduce by 5

# We need to add in a third embedding layer for genre
movie_genre = mx.symbol.Variable("movie_genre")
movie_genre = mx.symbol.Embedding(data=movie_genre, input_dim=20, output_dim=5) # Set to 5

y_true = mx.symbol.Variable("softmax_label")

nn = mx.symbol.concat(user, movie, movie_genre)
nn = mx.symbol.flatten(nn)
nn = mx.symbol.FullyConnected(data=nn, num_hidden=64)
nn = mx.symbol.Activation(data=nn, act_type='relu')
nn = mx.symbol.FullyConnected(data=nn, num_hidden=64)
nn = mx.symbol.Activation(data=nn, act_type='relu')
nn = mx.symbol.FullyConnected(data=nn, num_hidden=1)

y_pred = mx.symbol.LinearRegressionOutput(data=nn, label=y_true)

model = mx.module.Module(context=mx.cpu(), data_names=('user', 'movie', 'movie_genre'), symbol=y_pred)
model.fit(X_train, num_epoch=5, optimizer='adam', optimizer_params=(('learning_rate', 0.001),),
          eval_metric='rmse', eval_data=X_eval, batch_end_callback=mx.callback.Speedometer(batch_size, 250))

While it doesn't appear that using only the first genre has led to much improvement on this dataset, it demonstrates the types of things that one could do with the flexibility afforded by deep matrix factorization. In order to model all possible genres, perhaps one would create one embedding layer per genre that can either be 0 meaning the movie is not a part of that genre or a 1 meaning that it is. Alternatively, one could create a dense input vector that is encoded in the same way to replace the many embedding layers with a single input layer. Depending on the amount of information that is present for each user one may choose to add this type of structural regularization on the user axis as well.

## Conclusion

In this tutorial we investigated deep matrix factorization and showed how one would use Apache MXNet to implement these models. Since these types of matrix factorization models are commonly used to build recommender systems, we showed how to build models to predict user ratings of movies using the MovieLens 20M dataset. We saw that deep matrix factorization can natively take advantage of the recent advances in deep learning to learn more sophisticated models. In addition, deep matrix factorization can take advantage of the flexibility afforded by neural networks to allow more complicated inputs than traditional matrix factorization can handle that could yield additional insight into the task. Apache MXNet makes doing all of this simple, as with only about two dozen lines of code one can define both the model and how to train it. Since MXNet is built with distributed computing in mind it is easy to take advantage of those built-in features to scale this type of model up to massive datasets that can't fit in memory.

For further reading, the <a href="https://github.com/apache/incubator-mxnet/tree/master/example/recommenders">MXNet GitHub page</a> offers several tutorials on how to use MXNet to implement many types of recommender systems. Leo Dirac also has an <a href="https://www.youtube.com/watch?v=cftJAuwKWkA">excellent tutorial</a> on using MXNet for recommender systems that is worth watching.